In [34]:
import pandas as pd
import json
import nltk
from pymystem3 import Mystem
nltk.download("stopwords")
from nltk.corpus import stopwords
from string import punctuation

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np


from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\salim\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Using TensorFlow backend.


### Считываем данные

In [4]:
df = pd.read_excel('Datas/TestDataBase.xlsx')
df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163 entries, 0 to 162
Data columns (total 2 columns):
Заголовок статьи           163 non-null object
Портал самообслуживания    163 non-null object
dtypes: object(2)
memory usage: 2.6+ KB


### Создание JSON-файла для дальнейшей работы

In [5]:
data = {}

data['intents'] = []
for i,j in zip(df['Заголовок статьи'], df['Портал самообслуживания']):
    data['intents'].append({
        'tag':i,
        'patterns': [i],
        'responses': [j],
    })
    
with open('JSONforLemm.txt', 'w', encoding='UTF-8') as f:
    json.dump(data, f,ensure_ascii=False)

### Создание игнор-символов

In [6]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!',"''",'&','«','``','»',': &', 'quot', ')','(']

with open('JSONforLemm.txt', encoding='UTF-8') as json_file:
    intents = json.load(json_file)

### Токенизация 

In [13]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        w = nltk.word_tokenize(pattern, 'russian')
        words.extend(w)
        documents.append((w, intent['tag']))

        if intent['tag'] not in classes:
            classes.append(intent['tag'])     

### Лемматизация через бибилотеку от яндекса Pystem

In [15]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words and w not in punctuation]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

326 documents
161 classes ['An error has occurred, Подписание оферты', 'ID товара в СТЕ', 'Invalid params DTO: timeZone: Обязательно к заполнению.', 'Авторизация, неизвестная ошибка', 'Артикул в оферте', 'Архивирование оферты', 'Блокировка поставщика', 'Внесение изменений в отклоненную заявку на СТЕ', 'Внесение изменений в оферту', 'Вход по ЭП', 'Действие отменено пользователем (0x8010006e), при подписании оферты или контракта', 'Действия после отправки заявки на СТЕ', 'Действия после победы в Котировочной Сессии', 'Добавление файла в оферту по результатам Котировочной Сессии', 'Добавление файла в оферту.', 'Единицы измерения при формировании оферты', 'Заблокированный поставщик может публиковать оферты только по результатам КС', 'Загрузка представления, Произошла ошибка', 'Заказчик не выходит на связь после принятия услуг', 'Заключение контракта', 'Заполнение &quot;Нименование оферты&quot;', 'Заполнение заявки на изменение', 'Заполнение поля Регион регистрации организации.', 'Заполнени

### Записть лемматизированных слов в файлы типа .pkl для упрощения дальнейшего чтения

In [16]:
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))
# pickle.dump(words,open('words.pkl','wb'))
# pickle.dump(classes,open('classes.pkl','wb'))

# Шаблоны - Х
# пожелания - Y

In [32]:
print (len(classes), "classes \n")
print (len(words), " lemmatized words \n")
print(len(documents),'documents \n')

161 classes 

482  lemmatized words 

326 documents 



In [9]:
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [25]:
from numpy import random
# инициализация данных обучения
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # инициализация 'bag of words' 
    bag = []
    # список токенизированных слов для шаблона
    pattern_words = doc[0]
    # лемматизировать каждое слово - создать базовое слово, пытаясь представить родственные слова
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # создаем наш массив слов с 1, если в текущем шаблоне найдено совпадение слов
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output - это «0» для каждого тега и «1» для текущего тега (для каждого шаблона)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# перемешать 
random.shuffle(training)
training = np.array(training)
# X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])

In [26]:
print(pd.DataFrame(train_y).shape)
pd.DataFrame(train_y).head()

(326, 161)


,0,1,2,3,4,5,6,7,8,9,...,151,152,153,154,155,156,157,158,159,160
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
print(pd.DataFrame(train_x).shape)
pd.DataFrame(train_x).head()

(326, 482)


,0,1,2,3,4,5,6,7,8,9,...,472,473,474,475,476,477,478,479,480,481
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


## Обучение

In [36]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128*2, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Epoch 1/200
326/326 [==============================] - 0s 1ms/step - loss: 5.0861 - accuracy: 0.0031
Epoch 2/200
326/326 [==============================] - 0s 697us/step - loss: 5.0118 - accuracy: 0.0307
Epoch 3/200
326/326 [==============================] - 0s 725us/step - loss: 4.9361 - accuracy: 0.0429
Epoch 4/200
326/326 [==============================] - 0s 699us/step - loss: 4.8365 - accuracy: 0.0583
Epoch 5/200
326/326 [==============================] - 0s 1ms/step - loss: 4.6842 - accuracy: 0.0767
Epoch 6/200
326/326 [==============================] - 0s 1ms/step - loss: 4.4848 - accuracy: 0.1043
Epoch 7/200
326/326 [==============================] - 0s 1ms/step - loss: 4.1621 - accuracy: 0.1718
Epoch 8/200
326/326 [==============================] - 0s 1ms/step - loss: 3.9168 - accuracy: 0.2025
Epoch 9/200
326/326 [==============================] - 0s 1ms/step - loss: 3.4630 - accuracy: 0.2853
Epoch 10/200
326/326 [==============================] - 0s 724us/step - loss: 3.1696 

326/326 [==============================] - 0s 684us/step - loss: 0.1341 - accuracy: 0.9693
Epoch 81/200
326/326 [==============================] - 0s 818us/step - loss: 0.1492 - accuracy: 0.9571
Epoch 82/200
326/326 [==============================] - 0s 1ms/step - loss: 0.1054 - accuracy: 0.9755
Epoch 83/200
326/326 [==============================] - 0s 1ms/step - loss: 0.1635 - accuracy: 0.9540
Epoch 84/200
326/326 [==============================] - 0s 1ms/step - loss: 0.0985 - accuracy: 0.9755
Epoch 85/200
326/326 [==============================] - 0s 678us/step - loss: 0.1072 - accuracy: 0.9755
Epoch 86/200
326/326 [==============================] - 0s 1ms/step - loss: 0.1109 - accuracy: 0.9632
Epoch 87/200
326/326 [==============================] - 0s 1ms/step - loss: 0.1064 - accuracy: 0.9663
Epoch 88/200
326/326 [==============================] - 0s 642us/step - loss: 0.0967 - accuracy: 0.9755
Epoch 89/200
326/326 [==============================] - 0s 1ms/step - loss: 0.1030 - ac

326/326 [==============================] - 0s 1ms/step - loss: 0.0559 - accuracy: 0.9816
Epoch 160/200
326/326 [==============================] - 0s 1ms/step - loss: 0.0765 - accuracy: 0.9785
Epoch 161/200
326/326 [==============================] - 0s 1ms/step - loss: 0.0874 - accuracy: 0.9785
Epoch 162/200
326/326 [==============================] - 0s 1ms/step - loss: 0.0657 - accuracy: 0.9816
Epoch 163/200
326/326 [==============================] - 0s 1ms/step - loss: 0.0694 - accuracy: 0.9816
Epoch 164/200
326/326 [==============================] - 0s 1ms/step - loss: 0.0838 - accuracy: 0.9663
Epoch 165/200
326/326 [==============================] - 0s 1ms/step - loss: 0.1079 - accuracy: 0.9601
Epoch 166/200
326/326 [==============================] - 0s 1ms/step - loss: 0.0855 - accuracy: 0.9724
Epoch 167/200
326/326 [==============================] - 0s 1ms/step - loss: 0.0379 - accuracy: 0.9908
Epoch 168/200
326/326 [==============================] - 0s 1ms/step - loss: 0.0607 - a